In [1]:
import json
from pathlib import Path
from tqdm import tqdm_notebook
import psutil

import random
import numpy as np
import pandas as pd

import os
import time
import datetime


import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

In [2]:
bos_token = '<|start|>'
ans_token = ' [ANSWER] '
eos_token = '<|end|>'
pad_token = '<|pad|>'

def generate_text(subs, comments):
    texts = []
    for sub, comment in zip(subs, comments):
        text = bos_token + sub + ans_token + comment + eos_token
        texts.append(text)
    return texts

In [3]:
%%time
df = pd.read_csv(os.path.join('data', 'joined_data_AskReddit.csv.gz'), compression = 'gzip', sep = ';',)

CPU times: user 37.3 s, sys: 1.73 s, total: 39 s
Wall time: 39 s


In [4]:
df.shape

(9423894, 14)

In [5]:
df.author_comment.value_counts()

AutoModerator         81947
throwawayohyesitis    16033
KatyLiedTheBitch       4597
Gundam336B             4362
Flashy_Gardener        4172
                      ...  
DwindlingGravitas         1
Slut4GhostStories         1
Pinfield357               1
IMainJannaxxx             1
DrJuli                    1
Name: author_comment, Length: 1233063, dtype: int64

In [6]:
df[df.author_comment == 'AutoModerator'].head()

,link_id,id_comment,author_comment,total_awards_received_comment,text_comment,permalink,score_comment,subreddit,author_submission,text_submission,id_submission,total_awards_received_submission,score_submission,num_comments
65156,t3_a1xult,eatlgg5,AutoModerator,NaN,**Attention! [Serious] Tag Notice**\n\n* Jokes...,/r/AskReddit/comments/a1xult/serious_whats_a_g...,1.0,AskReddit,Brainiac03,[Serious] What's a good name for a travel blog?,a1xult,NaN,1,4
66618,t3_a1xa9a,eatgqq0,AutoModerator,NaN,**Attention! [Serious] Tag Notice**\n\n* Jokes...,/r/AskReddit/comments/a1xa9a/serious_redditors...,1.0,AskReddit,SMRNS2017,[Serious] Redditors who have been cheated on b...,a1xa9a,NaN,3,8
70416,t3_a1xxsa,eatm68d,AutoModerator,NaN,Hi there! Your post was removed because it use...,/r/AskReddit/comments/a1xxsa/have_yall_every_b...,1.0,AskReddit,damperfoot,"Have y'all every been friendzoned, be honest",a1xxsa,NaN,1,1
109957,t3_a1xy7v,eatma1b,AutoModerator,NaN,**PLEASE READ THIS MESSAGE IN ITS ENTIRETY BEF...,/r/AskReddit/comments/a1xy7v/worst_christmas_y...,2.0,AskReddit,Uatu_The_Watcher07,Worst Christmas you’ve ever had?,a1xy7v,NaN,1,1
129503,t3_a1xysw,eatmexc,AutoModerator,NaN,Hi there! Your post was removed because it use...,/r/AskReddit/comments/a1xysw/what_are_some_of_...,1.0,AskReddit,manutorrente,What are some of the funniest stories of your ...,a1xysw,NaN,1,1


In [7]:
df = df[df.author_comment != 'AutoModerator']

In [8]:
df['score_rank'] = df.groupby('id_submission').score_comment.rank(ascending = False)

<ipython-input-8-d38d98078de8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score_rank'] = df.groupby('id_submission').score_comment.rank(ascending = False)


In [9]:
df[df.id_submission == 'a1x406']

,link_id,id_comment,author_comment,total_awards_received_comment,text_comment,permalink,score_comment,subreddit,author_submission,text_submission,id_submission,total_awards_received_submission,score_submission,num_comments,score_rank
0,t3_a1x406,eatm38u,DomIstKrieg,NaN,Same. I would have missed some cool stuff.,/r/AskReddit/comments/a1x406/redditors_who_hav...,2.0,AskReddit,dorydoop,"Redditors who have come close to suicide, what...",a1x406,NaN,5,41,19.5
1,t3_a1x406,eatm4v8,dorydoop,NaN,Wow. That incredible. Did you get to know the ...,/r/AskReddit/comments/a1x406/redditors_who_hav...,2.0,AskReddit,dorydoop,"Redditors who have come close to suicide, what...",a1x406,NaN,5,41,19.5
2,t3_a1x406,eatm5z0,dorydoop,NaN,"So, are you doing better now?",/r/AskReddit/comments/a1x406/redditors_who_hav...,1.0,AskReddit,dorydoop,"Redditors who have come close to suicide, what...",a1x406,NaN,5,41,36.5
3,t3_a1x406,eatmdu8,TastyAssBiscuit,NaN,"Oh gosh, not a straightforward or easy questio...",/r/AskReddit/comments/a1x406/redditors_who_hav...,2.0,AskReddit,dorydoop,"Redditors who have come close to suicide, what...",a1x406,NaN,5,41,19.5
4,t3_a1x406,eatmgip,rollerderbydyke,NaN,Failed first attempt. Told myself I’d try agai...,/r/AskReddit/comments/a1x406/redditors_who_hav...,2.0,AskReddit,dorydoop,"Redditors who have come close to suicide, what...",a1x406,NaN,5,41,19.5
5,t3_a1x406,eatmi3t,DomIstKrieg,NaN,"Everytime i get a suicidal thought i think ""We...",/r/AskReddit/comments/a1x406/redditors_who_hav...,2.0,AskReddit,dorydoop,"Redditors who have come close to suicide, what...",a1x406,NaN,5,41,19.5
6,t3_a1x406,eatmqmo,jimmyablow09,NaN,When I was 23 I came home to find my wife with...,/r/AskReddit/comments/a1x406/redditors_who_hav...,3.0,AskReddit,dorydoop,"Redditors who have come close to suicide, what...",a1x406,NaN,5,41,4.5
7,t3_a1x406,eatmtg9,dorydoop,NaN,The power of a pinky promise. I'm sure your fr...,/r/AskReddit/comments/a1x406/redditors_who_hav...,2.0,AskReddit,dorydoop,"Redditors who have come close to suicide, what...",a1x406,NaN,5,41,19.5
8,t3_a1x406,eatmwt9,dorydoop,NaN,"So grateful for love at first sight, amirigt?",/r/AskReddit/comments/a1x406/redditors_who_hav...,2.0,AskReddit,dorydoop,"Redditors who have come close to suicide, what...",a1x406,NaN,5,41,19.5
9,t3_a1x406,eatmz54,SleepNowMyThrowaway,NaN,I did - we ended up dating :),/r/AskReddit/comments/a1x406/redditors_who_hav...,3.0,AskReddit,dorydoop,"Redditors who have come close to suicide, what...",a1x406,NaN,5,41,4.5


In [13]:
df = pd.merge(df, 
              df.groupby('id_submission').score_rank.min().reset_index().rename(
                  columns = {'score_rank': 'min_rank'}),
              how = 'left', on = 'id_submission')

In [15]:
df.head(2).T

,0,1
link_id,t3_a1x406,t3_a1x406
id_comment,eatm38u,eatm4v8
author_comment,DomIstKrieg,dorydoop
total_awards_received_comment,NaN,NaN
text_comment,Same. I would have missed some cool stuff.,Wow. That incredible. Did you get to know the ...
permalink,/r/AskReddit/comments/a1x406/redditors_who_hav...,/r/AskReddit/comments/a1x406/redditors_who_hav...
score_comment,2,2
subreddit,AskReddit,AskReddit
author_submission,dorydoop,dorydoop
text_submission,"Redditors who have come close to suicide, what...","Redditors who have come close to suicide, what..."


In [18]:
df = pd.merge(df.drop('min_rank', axis = 1), 
              df[df.score_rank > df.min_rank].groupby('id_submission').score_rank.min().reset_index().rename(
                  columns = {'score_rank': 'min_rank'}),
              how = 'left', on = 'id_submission')

In [20]:
tmp = df[df.score_rank <= df.min_rank]

In [21]:
tmp.shape

(1096538, 16)

In [22]:
texts = generate_text(tmp.text_submission, tmp.text_comment)

In [23]:
train_size = int(0.8 * len(texts))
test_size = len(texts) - train_size
train_dataset, test_dataset = random_split(texts, [train_size, test_size])

In [24]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2', 
                                          bos_token=bos_token, 
                                          eos_token=eos_token, 
                                          pad_token=pad_token,)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [25]:
class GPT2Dataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length=768):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []
        for txt in tqdm_notebook(txt_list):
            encodings_dict = tokenizer(txt, 
                                       truncation=True, 
                                       max_length=max_length, 
                                       padding="max_length")

            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
            if psutil.virtual_memory().percent > 95:
                break
    
    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx] 

In [26]:
%%time
dataset = GPT2Dataset(train_dataset, tokenizer, max_length=1024)

<ipython-input-25-7555c8d73ab3>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for txt in tqdm_notebook(txt_list):



CPU times: user 4min 57s, sys: 13.5 s, total: 5min 11s
Wall time: 5min 10s


In [27]:
%%time
val_dataset = GPT2Dataset(test_dataset, tokenizer, max_length=1024)

<ipython-input-25-7555c8d73ab3>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for txt in tqdm_notebook(txt_list):



CPU times: user 1min 12s, sys: 3.3 s, total: 1min 15s
Wall time: 1min 15s


In [28]:
batch_size = 2

train_dataloader = DataLoader(
            dataset,
            sampler = RandomSampler(dataset),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [29]:
model = GPT2LMHeadModel.from_pretrained("distilgpt2",)

In [30]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 768)

In [31]:
device = torch.device("cuda")
model.to(device)

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [32]:
epochs = 3
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

sample_every = 20000

optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

In [33]:
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
total_t0 = time.time()

training_stats = []


for epoch_i in range(0, epochs):
    #               Training
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in tqdm_notebook(enumerate(train_dataloader)):
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, 
                        labels = b_labels, 
                        attention_mask = b_masks, 
                        token_type_ids = None)

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(
                step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                bos_token_id=random.randint(1,30000),
                do_sample=True,   
                top_k=50, 
                max_length = 200,
                top_p=0.95, 
                num_return_sequences=1
            )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

        loss.backward()
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in tqdm_notebook(validation_dataloader):
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids, 
#                            token_type_ids=None, 
                             attention_mask = b_masks,
                            labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
    torch.save(model.state_dict(), 'distilgpt2_mln1train_ep' + str(epoch_i + 1) + '.ph')

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 3 ========
Training...


<ipython-input-34-6626bd345217>:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for step, batch in tqdm_notebook(enumerate(train_dataloader)):


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  Batch 20,000  of  438,615. Loss: 0.1842157244682312.   Elapsed: 1:21:47.
0:  bipartisanWhat are some of the best way to make money? [ANSWER] Make a difference and take a loan for it 


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


...

  Batch 280,000  of  438,615. Loss: 0.08846564590930939.   Elapsed: 19:11:11.
0:  NamWho's the next celebrity that you wish you could date? [ANSWER] Danny Devito for sure 


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  Batch 300,000  of  438,615. Loss: 0.13374905288219452.   Elapsed: 20:32:46.
0: IONWhat fictional characters would be actually really boring? [ANSWER] Oh yeah. Are you joking?


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


  Batch 320,000  of  438,615. Loss: 0.1083112359046936.   Elapsed: 21:54:58.
0:  glimpse[Serious] What’s a story that you have a hidden meaning for? [ANSWER] It’s not the first time, just the one time, but the last minute in the last year I was in love with my girlfriend, and she was at her friend’s house at the time. I was like, “hey man, you’re doing fine?” I say. She starts talking my girlfriend and then talking to me, just a couple months later. She comes to me before she is in a relationship. She’s also the opposite of me, and also the opposite of her brother/ sister. She’s also the perfect best at the same time, so she has a very special place. I’m just never close to anyone or anything like that. She’s a very big fan of my country but is always on their own.


In [231]:
sample = 'Which song gives you goosebumps?'
sample = bos_token + sample + ans_token
encodings_dict = tokenizer(sample, truncation=True, max_length=1024, )
input_ids = torch.tensor(encodings_dict['input_ids']).to(device)
greedy_output = model.generate(input_ids.unsqueeze(0),                             
                               max_length = 140,
                               num_beams = 1,
                               no_repeat_ngram_size = 2, 
                               num_return_sequences = 1,
                               early_stopping=True)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [232]:
for i, beam_output in enumerate(greedy_output):
    print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))
    print()

0: Which song gives you goosebumps? [ANSWER] 

I'm not sure if this counts but I've heard it a few times. I'm sure it's a good song.] The song "The Night Before" by The Night before the end of the night. ] I think it was a really good album. It's just a beautiful song, but it has a lot of songs that I can't really explain.
 fortune is a very good one. The lyrics are a bit too much. But it is so beautiful. And it makes me feel like I have a great time. So I feel so good about it.





In [229]:
sample = "What was a dumb belief you had as a kid?"
sample = bos_token + sample + ans_token
encodings_dict = tokenizer(sample, truncation=True, max_length=1024, )
input_ids = torch.tensor(encodings_dict['input_ids']).to(device)
greedy_output = model.generate(input_ids.unsqueeze(0),                             
                               max_length = 60,
                               num_beams = 10,
                               no_repeat_ngram_size = 2, 
                               num_return_sequences = 3,
                               early_stopping=True)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [230]:
for i, beam_output in enumerate(greedy_output):
    print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))
    print()

0: What was a dumb belief you had as a kid? [ANSWER]  “I’m an adult.” 

I used to believe that when I was younger, I thought it would be a good idea to have a baby.  



1: What was a dumb belief you had as a kid? [ANSWER]  “I’m an adult.” 

I used to believe that when I was younger, I thought it would be a good idea to have a child.  



2: What was a dumb belief you had as a kid? [ANSWER]  “I’m an adult.” 

I used to believe that when I was younger, I thought it would be a good idea to have a child.  


It was

